# import

In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from collections import Counter
import time, datetime, MeCab
import pandas as pd

# chromeDriver

In [2]:
#chromeをheadlessで起動
options = Options()
options.add_argument('--headless')
driver = webdriver.Chrome("/usr/local/bin/chromedriver",  chrome_options=options)
#driver = webdriver.Chrome("/usr/local/bin/chromedriver")
top_page = "https://news.yahoo.co.jp/"
dt_now = datetime.datetime.now()
driver.get(top_page)


<ipython-input-2-a990e741f5d2>:4: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome("/usr/local/bin/chromedriver",  chrome_options=options)


# ニュースリストの取得

In [3]:
df = pd.DataFrame(columns=['news_title','comment_url'])
#topicの選択
select_M_Topic = driver.find_element_by_link_text("トップ").click()
time.sleep(0.1)
select_S_Topic = driver.find_element_by_link_text("主要").click()
time.sleep(0.1)

#ニュース記事への移動
topPage = driver.current_url
#news[0].click()
for a in range(8):
#     print(a+1)
    section = driver.find_element_by_tag_name("section")
    news_list = section.find_elements_by_tag_name("li")
    mv_news = news_list[a].click()
    time.sleep(0.1)
    news_main = driver.find_element_by_tag_name("article")
    news_title = news_main.find_elements_by_tag_name("p")
    print(news_title[2].text)

    #コメントページへの移動
    try:
        news_comment_url = driver.find_element_by_class_name("news-comment-plugin").get_attribute("data-full-page-url")
#         print(news_comment_url)
        df.loc[a] = [news_title[2].text,news_comment_url]    
    except:
#         print('This news page have no comment.')
        df.loc[a] = [news_title[2],'This news page have no comment.']    
    finally:
        driver.get(topPage) 
        #time.sleep(1)

dt = dt_now.strftime('%Y%m%d_%H%M')
df.to_csv('NewsList/'+dt+'.csv')

緊急事態宣言、２月７日までの１カ月間に　百貨店にも時短要請
休業の「命令」違反業者に50万円以下の過料　特措法の政府原案が判明
政府　コロナ医療機関に支援金上乗せの方針
中国、感染対策に集中と釈明　ＷＨＯ調査団入国できず
「国や医師会に憤りを感じる。このままでは医療崩壊だけでなく“居酒屋崩壊”だ」緊急事態宣言の再発出を前に、厚労省の元医系技官が訴え
なぜ？『鴨川が真っ赤に染まる』京都市南区で異変…市職員や警察も駆けつけて原因調査
ラッパーのZeebra　コロナ陽性　軽症で「しばらくの間自宅療養となりました」
宮野真守　トップ声優から、枠にとらわれないエンターテイナーへ


# コメント情報の取得

In [4]:
df = pd.read_csv('NewsList/'+str(dt)+'.csv')
url = df['comment_url'][0]
title = df['news_title'][0]
driver.get(url)
print(title)

#コメント情報の取得

comment_boxes = []

#df = pd.DataFrame(columns=['number','日付','ユーザー名','コメント','good','bad'])
df = pd.DataFrame(columns=['number','ユーザー名','コメント'])
start = 1
end = 200
base_url = driver.current_url
page_url = '?page='
order_url = '&order=older'

iframe = driver.find_element_by_class_name("news-comment-plguin-iframe")
driver.switch_to_frame(iframe)
end = driver.find_element_by_class_name("pagenation")
pages = end.find_elements_by_class_name("rapidnofollow")
last_page = pages[7].text
driver.switch_to_default_content()
print(last_page)
for page in range(start,int(last_page)):
    
    driver.get(base_url+page_url+str(page)+order_url) 
    print(driver.current_url)
#    time.sleep(1)
    iframe = driver.find_element_by_class_name("news-comment-plguin-iframe")
    driver.switch_to_frame(iframe)
    
    comment_boxes = driver.find_elements_by_class_name("root")
    page_comment =0
    
    for comment_box in comment_boxes:
        

        
        page_comment +=1
        comment_number = str(page)+'-'+str(page_comment)
        
        #ユーザー名取得
        elem_name = comment_box.find_element_by_class_name("rapidnofollow")
        name = elem_name.text
        
        #コメント取得
        try:
            elem_comment = comment_box.find_element_by_class_name("cmtBody")
        except:
            elem_comment = comment_box.find_element_by_class_name("yjxComment")

        comment = elem_comment.text
        comment.strip("\n")
        
        #good数取得
        try:
            agree_box = comment_box.find_element_by_class_name("good")
            elem_agree = agree_box.find_element_by_class_name("userNum")
            agree = elem_agree.text
        except:
            pass

        #bad数取得
        try:
            disagree_box = comment_box.find_element_by_class_name("bad")
            elem_disagree = disagree_box.find_element_by_class_name("userNum")
            disagree = elem_disagree.text
        except:
            pass
        #df.loc[(page-1)*10+page_comment] = [comment_number,date, name, comment, agree, disagree]
        df.loc[(page-1)*10+page_comment] = [comment_number, name, comment]

df.to_csv('NewsComment/'+title+".csv")

20210107_0110
緊急事態宣言、２月７日までの１カ月間に　百貨店にも時短要請
260


<ipython-input-4-57d445069f9a>:21: DeprecationWarning: use driver.switch_to.frame instead
  driver.switch_to_frame(iframe)
<ipython-input-4-57d445069f9a>:25: DeprecationWarning: use driver.switch_to.default_content instead
  driver.switch_to_default_content()


https://news.yahoo.co.jp/articles/26460473ebd856c9cb8f261da71efdb7b4ba2489/comments?page=1&order=older


<ipython-input-4-57d445069f9a>:33: DeprecationWarning: use driver.switch_to.frame instead
  driver.switch_to_frame(iframe)


https://news.yahoo.co.jp/articles/26460473ebd856c9cb8f261da71efdb7b4ba2489/comments?page=2&order=older
https://news.yahoo.co.jp/articles/26460473ebd856c9cb8f261da71efdb7b4ba2489/comments?page=3&order=older
https://news.yahoo.co.jp/articles/26460473ebd856c9cb8f261da71efdb7b4ba2489/comments?page=4&order=older
https://news.yahoo.co.jp/articles/26460473ebd856c9cb8f261da71efdb7b4ba2489/comments?page=5&order=older
https://news.yahoo.co.jp/articles/26460473ebd856c9cb8f261da71efdb7b4ba2489/comments?page=6&order=older
https://news.yahoo.co.jp/articles/26460473ebd856c9cb8f261da71efdb7b4ba2489/comments?page=7&order=older
https://news.yahoo.co.jp/articles/26460473ebd856c9cb8f261da71efdb7b4ba2489/comments?page=8&order=older
https://news.yahoo.co.jp/articles/26460473ebd856c9cb8f261da71efdb7b4ba2489/comments?page=9&order=older
https://news.yahoo.co.jp/articles/26460473ebd856c9cb8f261da71efdb7b4ba2489/comments?page=10&order=older
https://news.yahoo.co.jp/articles/26460473ebd856c9cb8f261da71efdb7b4ba24

https://news.yahoo.co.jp/articles/26460473ebd856c9cb8f261da71efdb7b4ba2489/comments?page=81&order=older
https://news.yahoo.co.jp/articles/26460473ebd856c9cb8f261da71efdb7b4ba2489/comments?page=82&order=older
https://news.yahoo.co.jp/articles/26460473ebd856c9cb8f261da71efdb7b4ba2489/comments?page=83&order=older
https://news.yahoo.co.jp/articles/26460473ebd856c9cb8f261da71efdb7b4ba2489/comments?page=84&order=older
https://news.yahoo.co.jp/articles/26460473ebd856c9cb8f261da71efdb7b4ba2489/comments?page=85&order=older
https://news.yahoo.co.jp/articles/26460473ebd856c9cb8f261da71efdb7b4ba2489/comments?page=86&order=older
https://news.yahoo.co.jp/articles/26460473ebd856c9cb8f261da71efdb7b4ba2489/comments?page=87&order=older
https://news.yahoo.co.jp/articles/26460473ebd856c9cb8f261da71efdb7b4ba2489/comments?page=88&order=older
https://news.yahoo.co.jp/articles/26460473ebd856c9cb8f261da71efdb7b4ba2489/comments?page=89&order=older
https://news.yahoo.co.jp/articles/26460473ebd856c9cb8f261da71efd

https://news.yahoo.co.jp/articles/26460473ebd856c9cb8f261da71efdb7b4ba2489/comments?page=160&order=older
https://news.yahoo.co.jp/articles/26460473ebd856c9cb8f261da71efdb7b4ba2489/comments?page=161&order=older
https://news.yahoo.co.jp/articles/26460473ebd856c9cb8f261da71efdb7b4ba2489/comments?page=162&order=older
https://news.yahoo.co.jp/articles/26460473ebd856c9cb8f261da71efdb7b4ba2489/comments?page=163&order=older
https://news.yahoo.co.jp/articles/26460473ebd856c9cb8f261da71efdb7b4ba2489/comments?page=164&order=older
https://news.yahoo.co.jp/articles/26460473ebd856c9cb8f261da71efdb7b4ba2489/comments?page=165&order=older
https://news.yahoo.co.jp/articles/26460473ebd856c9cb8f261da71efdb7b4ba2489/comments?page=166&order=older
https://news.yahoo.co.jp/articles/26460473ebd856c9cb8f261da71efdb7b4ba2489/comments?page=167&order=older
https://news.yahoo.co.jp/articles/26460473ebd856c9cb8f261da71efdb7b4ba2489/comments?page=168&order=older
https://news.yahoo.co.jp/articles/26460473ebd856c9cb8f2

https://news.yahoo.co.jp/articles/26460473ebd856c9cb8f261da71efdb7b4ba2489/comments?page=239&order=older
https://news.yahoo.co.jp/articles/26460473ebd856c9cb8f261da71efdb7b4ba2489/comments?page=240&order=older
https://news.yahoo.co.jp/articles/26460473ebd856c9cb8f261da71efdb7b4ba2489/comments?page=241&order=older
https://news.yahoo.co.jp/articles/26460473ebd856c9cb8f261da71efdb7b4ba2489/comments?page=242&order=older
https://news.yahoo.co.jp/articles/26460473ebd856c9cb8f261da71efdb7b4ba2489/comments?page=243&order=older
https://news.yahoo.co.jp/articles/26460473ebd856c9cb8f261da71efdb7b4ba2489/comments?page=244&order=older
https://news.yahoo.co.jp/articles/26460473ebd856c9cb8f261da71efdb7b4ba2489/comments?page=245&order=older
https://news.yahoo.co.jp/articles/26460473ebd856c9cb8f261da71efdb7b4ba2489/comments?page=246&order=older
https://news.yahoo.co.jp/articles/26460473ebd856c9cb8f261da71efdb7b4ba2489/comments?page=247&order=older
https://news.yahoo.co.jp/articles/26460473ebd856c9cb8f2

In [107]:
df = pd.read_csv('NewsComment/'+title+'.csv')
df_parses = pd.DataFrame(columns=['出現単語','原型','品詞'])
wakati = MeCab.Tagger("-Owakati")
chasen = MeCab.Tagger("-Ochasen")
comment_list = df['コメント']
words = []
c=0
#for b in range(df.shape[0]):
comment = comment_list[0]
print(comment)
words += wakati.parse(comment).split()
for word in words:
    Word = chasen.parse(word).split()
    print(Word)
    if '助詞' in Word[3] or '記号' in Word[3]:
        continue
    else:
        df_parses.loc[z]=[Word[0],Word[2],Word[3]]
    c+=1
#print(Counter(word))
df_parses

数値基準を設けて解除しろや

いつまで匙加減やねん。。。
['数値基準', 'スウチキジュン', '数値基準', '名詞-固有名詞-一般', 'EOS']
['を', 'ヲ', 'を', '助詞-格助詞-一般', 'EOS']
['設け', 'モウケ', '設け', '名詞-一般', 'EOS']
['て', 'テ', 'て', '助詞-格助詞-連語', 'EOS']
['解除', 'カイジョ', '解除', '名詞-サ変接続', 'EOS']
['しろ', 'シロ', 'する', '動詞-自立', 'サ変・スル', '命令ｒｏ', 'EOS']
['や', 'ヤ', 'や', '助詞-終助詞', 'EOS']
['いつ', 'イツ', 'いつ', '名詞-代名詞-一般', 'EOS']
['まで', 'マデ', 'まで', '助詞-副助詞', 'EOS']
['匙加減', 'サジカゲン', '匙加減', '名詞-一般', 'EOS']
['や', 'ヤ', 'や', '助詞-終助詞', 'EOS']
['ねん', 'ネン', 'ねん', '助詞-終助詞', 'EOS']
['。', '。', '。', '記号-句点', 'EOS']
['。', '。', '。', '記号-句点', 'EOS']
['。', '。', '。', '記号-句点', 'EOS']


,出現単語,原型,品詞
0,数値基準,数値基準,名詞-固有名詞-一般
1,設け,設け,名詞-一般
2,解除,解除,名詞-サ変接続
3,しろ,する,動詞-自立
4,いつ,いつ,名詞-代名詞-一般
5,匙加減,匙加減,名詞-一般


In [111]:
for comment in comment_list:
    wordlist = wakati.parse(comment).split()
    markov = {}
    w1 = ""
    w2 = ""
    for word in wordlist:
        if w1 and w2:
            if (w1, w2) not in markov:
                markov[(w1, w2)]=[]
            markov[(w1, w2)].append(word)
        w1 = w2
        w2 = word
    print(markov)

{('数値基準', 'を'): ['設け'], ('を', '設け'): ['て'], ('設け', 'て'): ['解除'], ('て', '解除'): ['しろ'], ('解除', 'しろ'): ['や'], ('しろ', 'や'): ['いつ'], ('や', 'いつ'): ['まで'], ('いつ', 'まで'): ['匙加減'], ('まで', '匙加減'): ['や'], ('匙加減', 'や'): ['ねん'], ('や', 'ねん'): ['。'], ('ねん', '。'): ['。'], ('。', '。'): ['。']}
{('お', '尻'): ['しか'], ('尻', 'しか'): ['見え'], ('しか', '見え'): ['ませ'], ('見え', 'ませ'): ['ん'], ('ませ', 'ん'): ['が'], ('ん', 'が'): ['、'], ('が', '、'): ['中身'], ('、', '中身'): ['は'], ('中身', 'は'): ['？']}
{('春', 'から'): ['言わ'], ('から', '言わ'): ['れ'], ('言わ', 'れ'): ['て'], ('れ', 'て'): ['い', '医療'], ('て', 'い'): ['た'], ('い', 'た'): ['「'], ('た', '「'): ['とにかく'], ('「', 'とにかく'): ['PCR検査'], ('とにかく', 'PCR検査'): ['を'], ('PCR検査', 'を'): ['増やせ'], ('を', '増やせ'): ['ー'], ('増やせ', 'ー'): ['！'], ('ー', '！'): ['」'], ('！', '」'): ['「', 'っていう'], ('」', '「'): ['そんな'], ('「', 'そんな'): ['こと'], ('そんな', 'こと'): ['し'], ('こと', 'し'): ['たら'], ('し', 'たら'): ['治療', '、'], ('たら', '治療'): ['の'], ('治療', 'の'): ['必要'], ('の', '必要'): ['の'], ('必要', 'の'): ['ない'], ('の', 'ない'): ['無'], ('ない', '無'): 

{('二階', 'は'): ['そんなに'], ('は', 'そんなに'): ['権力者'], ('そんなに', '権力者'): ['なのか'], ('権力者', 'なのか'): ['?'], ('なのか', '?'): ['何故'], ('?', '何故'): ['そのよう'], ('何故', 'そのよう'): ['な'], ('そのよう', 'な'): ['事'], ('な', '事'): ['に'], ('事', 'に'): ['なっ'], ('に', 'なっ'): ['て'], ('なっ', 'て'): ['い'], ('て', 'い'): ['ます'], ('い', 'ます'): ['か'], ('ます', 'か'): ['?'], ('か', '?'): ['何', '疑問'], ('?', '何'): ['を'], ('何', 'を'): ['根拠'], ('を', '根拠'): ['に'], ('根拠', 'に'): ['1'], ('に', '1'): ['ヶ月'], ('1', 'ヶ月'): ['に'], ('ヶ月', 'に'): ['制限'], ('に', '制限'): ['する'], ('制限', 'する'): ['の'], ('する', 'の'): ['か'], ('の', 'か'): ['?'], ('?', '疑問'): ['だらけ'], ('疑問', 'だらけ'): ['です'], ('だらけ', 'です'): ['黒い'], ('です', '黒い'): ['暗い'], ('黒い', '暗い'): ['闇'], ('暗い', '闇'): ['でしょ'], ('闇', 'でしょ'): ['う'], ('でしょ', 'う'): ['か']}
{('ここ', 'まで'): ['制限'], ('まで', '制限'): ['（'], ('制限', '（'): ['国民'], ('（', '国民'): ['の'], ('国民', 'の'): ['我慢'], ('の', '我慢'): ['）'], ('我慢', '）'): ['し'], ('）', 'し'): ['て'], ('し', 'て'): ['おい'], ('て', 'おい'): ['て'], ('おい', 'て'): ['、'], ('て', '、'): ['春節'], ('、', '春

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [99]:
driver.quit()